In [1]:
import glob
import itertools
import os
import random

import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sn

plt.style.use("ggplot")

# Explore LOQs

See e-mail from Øyvind K received 28.11.2024.

## 1. Get historic data from Vannmiljø

In [2]:
# Map parameters to historic LOQs in Kjetil's Word doc
pars_loq_dict = {
    "PH": None,
    "ALK": 0.02,
    "KOND": 0.5,
    "TOC": 0.5,
    "CA": 0.15,
    "MG": 0.1,
    "K": 0.05,
    "NA": 0.3,
    "CL": 0.45,
    "SO4": 0.3,
    "N-NO3": 10,
    "SIO2": 50, # mg/l in Word; ug/l in VM
    "N-TOT": 25,
    "P-TOT": 2,
    "AL": None,
    "RAL": 5,
    "ILAL": 5,
    "P-PO4": 2,
    "N-NH4": 2,
    "AS": 0.05,
    "PB": 0.15,
    "CD": 0.02,
    "CR": 0.1,
    "CU": 0.1,
    "NI": 0.5,
    "ZN": 2,
    "HG": 0.02,
    "AG": 0.01,
    "STS": 2,
    "TURB": 0.1,
    "DOC": 0.5,
}

# Data for ~past decade
st_yr, end_yr = 2013, 2023

In [3]:
# Get data from Vannmiljø API
pars = list(pars_loq_dict.keys())
tiltak_stn_df = pd.read_excel(r"../data/active_stations_2020.xlsx", sheet_name="data")
filter_dict = {
    "WaterLocationCodeFilter": tiltak_stn_df["vannmiljo_code"].tolist(),
    "FromDateSamplingTime": f"{st_yr}-01-01",
    "ToDateSamplingTime": f"{end_yr}-12-31",
    "ParameterIDFilter": pars,
    "FromRegDate": "1900-01-01",
}
vm_df = nivapy.da.post_data_to_vannmiljo("GetRegistrations", data=filter_dict)
vm_df = vm_df.query("MediumID == 'VF'")
vm_df = vm_df.query("ActivityID in ('KALK', 'KALL')")

# Split station details from chem and tidy
stn_col_dict = {
    "WaterLocationCode": "station_code",
    "Name": "station_name",
    "CoordX": "utm33_east",
    "CoordY": "utm33_north",
    "WaterCategory": "type",
    "VassdragsomradeID": "vassom_id",
    "Vassdragsomrade": "vassom_name",
    "WaterBodyID": "waterbody_id",
    "WaterBody": "waterbody_name",
    "VannomradeID": "vannom_id",
    "Vannomrade": "vannom_name",
    "VannregionID": "vannreg_id",
    "Vannregion": "vannreg_name",
    "Fylke": "fylke",
    "Kommune": "kommune",
}
stn_df = (
    vm_df[stn_col_dict.keys()].copy().rename(columns=stn_col_dict).drop_duplicates()
)
assert stn_df["station_code"].is_unique

wc_col_dict = {
    "WaterLocationCode": "station_code",
    "ActivityID": "activity_id",
    "ActivityName": "activity_name",
    "Employer": "employer",
    "Contractor": "contractor",
    "SamplingTime": "sampling_date",
    "UpperDepth": "upper_depth",
    "LowerDepth": "lower_depth",
    "ParameterID": "parameter",
    "ValueOperator": "flag",
    "RegValue": "value",
    "Unit": "unit",
}
vm_df = vm_df[wc_col_dict.keys()].copy().rename(columns=wc_col_dict)
vm_df.head()

,station_code,activity_id,activity_name,employer,contractor,sampling_date,upper_depth,lower_depth,parameter,flag,value,unit
0,027-28435,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,NIVA,2013-01-07T00:00:00,NaN,NaN,ALK,=,0.053,mmol/l
1,027-28435,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,NIVA,2013-02-04T00:00:00,NaN,NaN,ALK,=,0.066,mmol/l
2,027-28435,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,NIVA,2013-03-04T00:00:00,NaN,NaN,ALK,=,0.084,mmol/l
3,027-28435,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,NIVA,2013-04-01T00:00:00,NaN,NaN,ALK,=,0.084,mmol/l
4,027-28435,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,NIVA,2013-05-06T00:00:00,NaN,NaN,ALK,=,0.066,mmol/l


## 2. Summarise LOQ values reported in Vannmiljø

In [4]:
# Print reported LOQ values for each par
print(
    f"In vannmiljø, the following LOQ values have been reported for each parameter between {st_yr} and {end_yr}:\n"
)
for par in sorted(pars):
    par_loq_df = vm_df.query("(parameter == @par) and (flag == '<')")
    if not par_loq_df.empty:
        # Check all units are the same
        units = par_loq_df["unit"].unique().tolist()
        assert len(units) == 1
        unit = units[0]
        par_unit = f"{par} ({unit})"
        loqs = sorted(par_loq_df["value"].unique().tolist())
        print(f"{par_unit:<15}", loqs)
    else:
        print(f"{par:<15}", "No data reported at LOQ.")

In vannmiljø, the following LOQ values have been reported for each parameter between 2013 and 2023:

AG (µg/l)       [0.01, 0.02, 0.05, 0.1]
AL              No data reported at LOQ.
ALK (mmol/l)    [0.01, 0.02, 0.03]
AS (µg/l)       [0.02, 0.2]
CA (mg/l)       [0.02, 0.1, 0.14]
CD (µg/l)       [0.003, 0.02, 0.03, 0.05, 0.1]
CL (mg/l)       [0.1, 0.5]
CR (µg/l)       [0.05, 0.1, 0.5]
CU (µg/l)       [0.05, 0.1, 0.2, 0.5]
DOC             No data reported at LOQ.
HG (µg/l)       [0.005, 0.01, 0.02, 0.05]
ILAL (µg/l Al)  [5.0]
K (mg/l)        [0.02, 0.05, 0.5, 0.91, 1.1]
KOND (mS/m)     [0.05, 0.1, 1.0]
MG (mg/l)       [0.2]
N-NH4 (µg/l N)  [2.0, 3.0]
N-NO3 (µg/l N)  [1.0, 1.6, 2.0, 5.0, 10.0]
N-TOT (µg/l N)  [10.0, 20.0]
NA (mg/l)       [0.5]
NI (µg/l)       [0.05, 0.5, 1.0]
P-PO4 (µg/l P)  [2.0]
P-TOT (µg/l P)  [0.4, 1.0, 2.0, 20.0]
PB (µg/l)       [0.01, 0.05, 0.1, 0.2, 0.5, 1.0]
PH              No data reported at LOQ.
RAL (µg/l Al)   [5.0]
SIO2 (µg/l Si)  [13.0, 20.0, 23.0, 51.0]
SO4 

## 3. Proportions of values at or below LOQ for each parameter

In [5]:
# Count the proportion of values for each par reported at the LOQ
loq_cnt_df = (
    vm_df.groupby(["parameter", "flag"])
    .size()
    .reset_index(name="n_loq")
    .query("flag == '<'")
)
tot_cnt_df = vm_df.groupby(["parameter"]).size().reset_index(name="n_total")
loq_cnt_df = pd.merge(tot_cnt_df, loq_cnt_df, how="left", on="parameter")
loq_cnt_df["n_loq"] = loq_cnt_df["n_loq"].fillna(0).astype(int)
loq_cnt_df["pct_loq"] = (100 * loq_cnt_df["n_loq"] / loq_cnt_df["n_total"]).round(1)
loq_cnt_df = loq_cnt_df.sort_values("pct_loq", ascending=False).reset_index(drop=True)
loq_cnt_df = loq_cnt_df[["parameter", "n_loq", "n_total", "pct_loq"]]
loq_cnt_df

,parameter,n_loq,n_total,pct_loq
0,HG,527,531,99.2
1,AG,474,480,98.8
2,STS,367,530,69.2
3,P-PO4,100,215,46.5
4,CR,210,523,40.2
5,CD,206,532,38.7
6,NI,195,531,36.7
7,P-TOT,2605,8381,31.1
8,PB,141,532,26.5
9,N-NH4,137,533,25.7


## 4. Percentiles corresponding to the specificed LOQ

In [6]:
# Get the percentile corresponding to the LOQ specified in the Word document
high_loq_pars = []
for par, loq in pars_loq_dict.items():
    if loq:
        par_df = vm_df.query("parameter == @par").copy()

        # Check all units are the same
        units = par_df["unit"].unique().tolist()
        assert len(units) == 1
        unit = units[0]
        par_unit = f"{par} ({unit})"
        loq_unit = f"{loq} ({unit})"

        # Calculate percentile
        values = par_df["value"].values
        percentile = 100 * np.sum(values <= loq) / len(values)

        if percentile > 5:
            high_loq_pars.append(par)

        print(f"{par_unit:<15} LOQ of {loq_unit:<15} corresponds to P{percentile:.1f}.")
    else:
        print(f"{par:<15} No LOQ defined.")

print("\n\nThe specified LOQs for the following parameters are above P05:\n")
print(high_loq_pars)

PH              No LOQ defined.
ALK (mmol/l)    LOQ of 0.02 (mmol/l)   corresponds to P1.4.
KOND (mS/m)     LOQ of 0.5 (mS/m)      corresponds to P0.2.
TOC (mg/l C)    LOQ of 0.5 (mg/l C)    corresponds to P1.6.
CA (mg/l)       LOQ of 0.15 (mg/l)     corresponds to P1.9.
MG (mg/l)       LOQ of 0.1 (mg/l)      corresponds to P1.5.
K (mg/l)        LOQ of 0.05 (mg/l)     corresponds to P1.1.
NA (mg/l)       LOQ of 0.3 (mg/l)      corresponds to P0.2.
CL (mg/l)       LOQ of 0.45 (mg/l)     corresponds to P0.5.
SO4 (mg/l)      LOQ of 0.3 (mg/l)      corresponds to P1.2.
N-NO3 (µg/l N)  LOQ of 10 (µg/l N)     corresponds to P3.9.
SIO2 (µg/l Si)  LOQ of 50 (µg/l Si)    corresponds to P1.5.
N-TOT (µg/l N)  LOQ of 25 (µg/l N)     corresponds to P0.2.
P-TOT (µg/l P)  LOQ of 2 (µg/l P)      corresponds to P39.4.
AL              No LOQ defined.
RAL (µg/l Al)   LOQ of 5 (µg/l Al)     corresponds to P1.5.
ILAL (µg/l Al)  LOQ of 5 (µg/l Al)     corresponds to P5.5.
P-PO4 (µg/l P)  LOQ of 2 (µg/l P)  

## 5. Percentiles for each parameter

In [7]:
# Percentiles for each parameter
percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]

# Create a new dataframe to store the percentiles
pct_df = pd.DataFrame()

# Calculate the percentiles for each parameter
for parameter in vm_df["parameter"].unique():
    values = vm_df[vm_df["parameter"] == parameter]["value"]
    percentile_values = np.percentile(values, percentiles)
    pct_df = pd.concat(
        [
            pct_df,
            pd.DataFrame([percentile_values], columns=percentiles, index=[parameter]),
        ]
    )

# Transpose the result dataframe to have one row per parameter and columns as percentiles
pct_df.columns = [f"{p}th" for p in percentiles]
pct_df

,1th,5th,10th,25th,50th,75th,90th,95th,99th
ALK,0.019000,0.030000,0.03400,0.04300,0.0570,0.07375,0.091,0.11000,0.15395
CA,0.120000,0.230000,0.31000,0.53700,0.8320,1.30000,1.880,2.52000,6.34000
ILAL,5.000000,5.000000,7.50000,14.00000,27.0000,48.00000,72.000,89.00000,121.61000
KOND,0.760000,1.000000,1.19000,1.56000,2.2000,3.00000,3.990,4.69000,6.88000
PH,4.950000,5.240000,5.43000,5.80000,6.1300,6.40000,6.600,6.75000,7.10000
RAL,5.000000,9.200000,13.00000,23.00000,41.0000,67.00000,104.000,127.00000,170.00000
TOC,0.470000,0.750000,0.98400,1.60000,2.7000,4.60000,6.500,7.90000,12.00000
SIO2,30.000000,161.000000,241.50000,360.00000,542.0000,823.00000,1159.000,1396.75000,2077.40000
SO4,0.270000,0.500000,0.58000,0.76000,1.0000,1.40000,1.900,2.30000,3.50000
P-TOT,1.000000,2.000000,2.00000,2.00000,3.0000,5.00000,9.000,12.00000,26.00000
